### MechInterp

In [ ]:
def hook_func(act, hook):
    act[:, 1, :] = clean_acts[hook.name][:, 1, :]

In [ ]:
model.run_with_hooks(
    corrupted_tokens,
    fwd_hooks=[(hook_name, hook_func)]
)

In [ ]:
from plotpl

In [ ]:
embeddings = model.embed(tokens) + model.pos_embed(tokens)

In [ ]:
residual = embeddings

In [ ]:
for block in model.blocks:
    residual = block(residual)

In [ ]:
residual = model.ln_final(residual)

In [ ]:
logits = model.unembed(residual)

In [ ]:
corrupted_prompt = "When John and Mary went to the shops, Mary gave the bag to"

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)

In [ ]:
corrupted_tokens = model.to_tokens(corrupted_tokens)

In [ ]:
clean_logits, clean_activations = model(clean_tokens)

In [ ]:
n_tokens = clean_tokens.shape[-1]

In [ ]:
def patch_act(
    corrupted_activations,
    hook,
    layer,
    position,
    clean_activations
)

In [ ]:
from functools import partial

In [ ]:
from transformer_lens.utils import get_act_name

In [ ]:
for layer in range(n_layers):
    for position in range(n_tokens):
        hook_func = partial(patch_act, position=position, clean_activations=clean_activations)
        hook_name = get_act_name("resid_pre", layer)
        
        patched_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(hook_name, hook_func)]
        )

In [ ]:
step 1: create two prompts
step 2: record act of clean prompt
step 3: iterative

In [ ]:
correct_token = model.to_single_token(" John")

In [ ]:
incorrect_token = model.to_single_token(" Mary")

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)

In [ ]:
n_tokens = clean_tokens.shape[-1]

In [ ]:
from functools import partial

In [ ]:
def patch_act(
    corrupted_activations,
    hook,
    position,
    clean_activations
):
    corrupted_activations[:, position, :] = clean_activations[hook.name][:, position, :]
    return corrupted_activations

In [ ]:
def compute_logit_diff(logits):
    last_token_logits = logits[:, -1, :]
    correct_logit = logits[:, corrct_]

In [ ]:
for layer in range(n_layers):
    for position in range(n_tokens):
        hook_func = partial(patch_act, position, clean_activations=clean_activations)
        hook_name = get_act_name("resid_pre", layer)
        patched_logits = model.run_with_hooks(
            hook_func,
            fwd_hooks=[(hook_name, hook_func)]
        )
        

In [ ]:
from einops import einsum

In [ ]:
interference = einsum(
    W, W,
    "batch feature1 dim, batch feature2 dim -> batch feature1 feature2"
)

### Engineering

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class VocabParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        world_size = torch.distributed.get_world_size()
        
        self.num_embeddings = num_embeddings
        self.embedding_dim_per_rank = embedding_dim // world_size
        
        self.embedding = nn.Embedding(
            self.num_embeddings,
            self.embedding_dim_per_rank
        )
        self.start_vocab_idx, self.end_vocab_idx = self.get_range_vocab_idx(self.embedding_dim_per_rank)
    
    def get_range_vocab_idx(self, embedding_dim_per_rank):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        
        start_idx = rank * embedding_dim_per_rank
        end_idx = start_idx + embedding_dim_per_rank
        
        return start_idx, end_idx
    
    def forward(self, tokens):
        mask = (tokens < self.start_vocab_idx) | (tokens >= self.end_vocab_idx)
        tokens = tokens - self.start_vocab_idx
        tokens[mask] = 0.
        
        embedding_parallel = self.embedding_dim_per_rank(tokens)
        mask_idx = torch.where(tokens == False)[1]
        embedding_parallel[mask_idx] = 0.
        
        torch.distributed.all_reduce(embedding_parallel)
        return em

In [ ]:
sorted_params = sorted(
    param_list,
    key=param_list.numel(),
    reverse=True
)

In [ ]:
numel_per_rank = [0 for _ in range(world_size)]
param_per_rank = [[] for _ in range(world_size)]

In [ ]:
for param in sorted_param:
    rank_to_go = numel_per_rank.index(min(numel_per_rank))
    param_per_rank[rank_to_go].append(param)
    numel_per_rank[rank_to_go] += param.numel()

In [ ]:
step 1: scale the loss
step 2: compute the gradient with respect to the scaled loss
step 3: accmulate the gradient with the previous accumulatied gradient
step 4: if update...

In [ ]:
register > cached memory > main memory > hard > backup

In [ ]:
stream1 = torch.cuda.Stream()
stream2 = torch.cuda.Stream()

In [ ]:
import time

In [ ]:
start_time = time.time()

with torch.cuda.stream(stream1):
    operation_a()
    
with torch.cuda.stream(stream2):
    operation_b()
    
torch.cuda.syncro

In [ ]:
ranks = [0, 1, 3, 6]

In [ ]:
rank = torch.distributed.get_rank()

In [ ]:
group = None

In [ ]:
if rank in ranks:
    group = torch.distributed.new_group(ranks)

In [ ]:
if group != None:
    torch.distributed.broadcast(x, src=0, group=group)

In [ ]:
step 1: determine checkpoint
step 2: compute the activation and only save the checkpoint
step 3: compute the gradient start from the checkpoi

In [ ]:
for i in range(num_tensor_model_parallel_groups):
    ranks = range(
        (i*tensor_model_parallel_size),
        (i*tensor_model_parallel_size)
    )
    
    

In [ ]:
new > ready > running > blocked > terminated

In [ ]:
step 1: determine checkpoint location
step 2: calculate the activation and save the acti checkpoint
step 3: delete the previous activation
step 3: 

In [ ]:
new > ready > running > blocked > terminated

### ML Engineering

In [ ]:
from functools import lru_cache

In [ ]:
from pydantic import BaseModel, validator

In [ ]:
class User(BaseModel):
    user_id: int
    username: str
    
    @validator("username")
    def uppercase(cls, value):
        return value.upper()

In [ ]:
from metaflow import FlowSpec, step, conda

In [ ]:
class TrainingFlow(FlowSpec):
    @step
    def start(self):
        self.next(self.train)
    
    @conda(python="3.11.0", libraries={"pytorch": "2.0.0"})
    @step
    def train(self):
        train()
        self.next(self.end)
    
    @step
    def end(self):
        pass

### AI

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from torch.profiler import profile, ProfilerActivity, schedule

In [ ]:
with profile(
    activities=[ProfilerActivity.CPU],
    schedule=schedule()
)

In [ ]:
import jax

In [ ]:
jax.tree_map(square, xs)

In [ ]:
tokenizer.add_special_tokens(SPECIAL_TOKENS)

In [ ]:
model.resize_token

In [ ]:
from einops import pack

In [ ]:
pack([image_rgb, image_depth], "")

In [ ]:
from einops import pack

In [ ]:
stacked_images = pack([image_rgb, image_depth], "h w *")

In [ ]:
correct_token = model.to_single_token(" John")

In [ ]:
incorrect_token = model.to_single_token(" Mary")

In [ ]:
clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

In [ ]:
clean_logits, clean_activations = model.run_with_cache(clean_tokens)

In [ ]:
def patch_it(
    corrupted_activations,
    hook,
    position,
    clean_activations
):
    corrupted_activations[:, position, :] = clean_activations[:, position, :]
    return corrupted_activations

In [ ]:
n_positions = clean_tokens.shape[-1]

In [ ]:
from functools import partial
from transformer_lens import utils

In [ ]:
def logit_diff(logits, correct_token, incorrect_token):
    last_token_logits = logits[:, -1, :]
    correct_l = last_token_logits[:, correct_token]
    incorrect_token = last_token_logits[:, incorrect_token]
    diff = 

In [ ]:
for layer in range(n_layers):
    for position in range(n_positions):
        hook_func = partial(patch_it, position=position, clean_activations=clean_activations)
        hook_name = utils.get_act_name("resid_pre", layer)
        corrupted_logits = model.run_with_hooks(
            corrupted_tokens,
            fwd_hooks=[(hook_name, hook_func)]
        )
        

In [ ]:
def probability_score(image_embedding, text_embedding):
    image_norm = image_embedding.norm(dim=-1)
    image_embedding = image_embedding / image_norm
    text_norm = text_embedding.norm(dim=-1)
    text_embedding = text_embedding / norm
    
    similarities = image_embedding @ text_embedding.T
    probs = F.softmax(similarities, dim=-1)